In [ ]:
%pip install pandas pinecone-client sentence-transformers openai

In [ ]:
%pip install openai

In [ ]:
import pandas as pd
import openai
import pinecone
from tqdm import tqdm
from openai import OpenAI


client = OpenAI(

)

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

episodes = pd.read_csv('Job_Placement_Data.csv')

# Features to process
features = ['ssc_percentage', 'ssc_board', 'hsc_percentage', 'work_experience', 'emp_test_percentage']
label = 'status'

# Convert 'work_experience' to string
episodes['work_experience'] = episodes['work_experience'].astype(str)

# Strip whitespace from string features
string_features = ['ssc_board', 'work_experience', 'status']
episodes[string_features] = episodes[string_features].apply(lambda x: x.str.strip())

# Remove any records with null values
episodes = episodes.dropna(subset=features + [label])

# Combine text columns into a single text field for embedding
episodes['combined_text'] = episodes[features].astype(str).agg(' '.join, axis=1)

# Generate embeddings for the combined text
episodes['embeddings'] = episodes['combined_text'].apply(lambda x: get_embedding(x))
import pinecone

# Initialize Pinecone
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
    api_key="452112a5-f4cc-45f0-a97e-0a17c17d3eb3"
)

# Create an index if not already created
index_name = 'garudahacks'
# if index_name not in pc.list_indexes():
#     pc.create_index(
#         name="garudahacks",
#         dimension=1536,
#         metric="cosine",
#         spec=ServerlessSpec(
#           cloud="aws",
#           region="us-east-1"
#     ))  # Assuming 384-dimensional embeddings
index = pc.Index(index_name)

# Prepare vectors to upsert
vectors_to_upsert = [
    (str(i), embedding, {"text": text, "label": label})
    for i, (embedding, text, label) in enumerate(zip(episodes['embeddings'], episodes['combined_text'], episodes['status']))
]

# for x in vectors_to_upsert:
#   print(x)

# # Upsert vectors into Pinecone
index.upsert(vectors=vectors_to_upsert, namespace="jobs")

{'upserted_count': 215}